In [19]:
import pandas as pd
import numpy as np
import os

from os.path import join as oj

In [11]:
root_folder = r'/Users/yang/Documents/Wilbrecht_Lab/sleap_video/preprocessed_data'
combined_df = pd.read_csv(oj(root_folder, 'combined_df.csv'))

In [12]:
print(combined_df.shape)
combined_df.head()

(1639209, 38)


,Head x,Head y,Neck x,Neck y,Torso x,Torso y,Tailhead x,Tailhead y,warped Head x,warped Head y,...,Head velocity,Neck velocity,Torso velocity,Tailhead velocity,Head acceleration,Neck acceleration,Torso acceleration,Tailhead acceleration,animal,session
0,192.409988,28.869892,195.444916,23.708437,197.048996,11.929043,203.344009,6.518982,274.010981,130.213296,...,0.000000,0.000000,0.000000,0.000000,5552.485839,5514.094622,5548.340616,2316.800553,RRM028,Day141
1,191.536728,36.181351,192.733429,28.968243,196.683792,16.905258,201.088638,7.058071,274.622855,124.074284,...,185.082861,183.803154,184.944687,77.226685,53.563670,157.500350,259.292766,2031.850638,RRM028,Day141
2,191.296310,41.741261,192.233994,36.576694,195.877075,24.634306,199.839661,12.491142,275.255325,117.877533,...,186.868317,189.053166,193.587779,144.955040,342.280183,140.446969,227.788654,1755.017224,RRM028,Day141
3,191.317017,48.960808,191.961182,43.915081,195.264984,32.221252,197.228607,20.520767,276.065172,111.318082,...,198.277656,193.734731,201.180735,203.455614,369.855782,146.696497,72.693587,-43.579525,RRM028,Day141
4,188.718781,57.093216,191.541870,49.983654,193.036667,39.939732,197.111298,27.591192,277.095983,104.373967,...,210.606182,198.624615,203.603854,202.002963,67.972904,253.944027,-28.426736,-315.863919,RRM028,Day141


In [13]:
def compare_decisions(df):
    """

    """
    current_trial_num = np.nan
    current_bonsai_decision = np.nan
    current_decision = np.nan
    bonsai_decision = np.nan
    decision = np.nan
    
    bonsai_decision_list = []
    decision_list = []
    animal_list = []
    session_list = []
    trial_list = []

    for index, row in df.iterrows():
        bonsai_decision = row['label']
        decision = row['decision']
        trial_num = row['trial']
        animal = row['animal']
        session = row['session']
        
        if trial_num != current_trial_num: # End of current trial and start of a new trial 
            if current_bonsai_decision == 'collection':
                current_bonsai_decision = 'ACC'
            if current_decision != current_bonsai_decision:
                bonsai_decision_list.append(current_bonsai_decision)
                decision_list.append(current_decision)
                animal_list.append(animal)
                session_list.append(session)
                trial_list.append(current_trial_num)

            current_bonsai_decision = np.nan    
            current_decision = np.nan
            current_trial_num = trial_num 
              
        if not pd.isna(decision):
            current_decision = decision    
        if not pd.isna(bonsai_decision):
            current_bonsai_decision = bonsai_decision
    
    result_df = pd.DataFrame({
        'bonsai decision': bonsai_decision_list,
        'decision': decision_list,
        'animal': animal_list,
        'session': session_list,
        'trial': trial_list
    })
    
    return result_df

In [14]:
different_trials = compare_decisions(combined_df)

In [18]:
#different_trials.to_csv(oj(root_folder, 'different_trials.csv'), index=False)
print(different_trials.shape)
different_trials.head()

(2272, 5)


,bonsai decision,decision,animal,session,trial
0,NaN,NaN,RRM028,Day141,NaN
1,quit,REJ,RRM028,Day141,20.0
2,quit,REJ,RRM028,Day141,66.0
3,quit,REJ,RRM028,Day141,85.0
4,quit,REJ,RRM028,Day141,97.0


In [17]:
ACC_REJ_trials = different_trials[
    ((different_trials['bonsai decision'] == 'ACC') & (different_trials['decision'] == 'REJ')) |
    ((different_trials['bonsai decision'] == 'REJ') & (different_trials['decision'] == 'ACC'))
]

quit_REJ_trials = different_trials[
    (different_trials['bonsai decision'] == 'quit') & (different_trials['decision'] == 'REJ')
]
print(len(ACC_REJ_trials))
print(len(quit_REJ_trials))

33
1912


In [22]:
a = different_trials[
    different_trials['bonsai decision'] == np.nan
]
a

,bonsai decision,decision,animal,session,trial


In [24]:
# Get unique combinations of decisions that exist in the DataFrame
existing_combinations = different_trials[['bonsai decision', 'decision']].drop_duplicates()

# Convert to list of tuples
existing_combinations_list = [tuple(x) for x in existing_combinations.to_records(index=False)]

# Display the combinations
print(existing_combinations_list)

[(nan, nan), ('quit', 'REJ'), ('T_Entry', 'REJ'), ('T_Entry', 'quit'), ('T_Entry', nan), ('REJ', 'quit'), ('outcome', 'quit'), ('tone_onset', 'REJ'), ('quit', 'ACC'), ('ACC', 'REJ'), ('ACC', 'quit'), ('ACC', 'T-Entry'), ('T_Entry', 'T-Entry'), ('outcome', 'REJ'), ('ACC', nan), ('tone_onset', 'ACC'), ('outcome', 'ACC'), ('tone_onset', 'T-Entry'), ('tone_onset', 'quit'), ('T_Entry', 'ACC'), (nan, 'quit')]
